# CSE4020 Activity 3
Grant Butler | [gbutler2020@my.fit.edu](mailto:gbutler2020@my.fit.edu)

<u>Question 1.</u>

Imagine that the bank has received a portion of a government bailout totaling 10 million dollars. One requirement for banks receiving this bailout money is that they remain their current size (ie., large branches are branches with assets >= 3 million dollars and small branches have assets < 3 million dollars). The CEO of the bank, Nate Richmond, would like to divide this bailout money equally among the small branches at the bank.

In [1]:
# - - - - - - - - - #
# connecting to RDS #
# - - - - - - - - - #

# imports
import json
import mysql.connector as connector

# loading credentials from json
f = open("rds_endpoint.json")
creds = json.load(f)

# making dict of credentials to log into the RDS server
config = {
    'user': creds["username"],
    'password': creds["password"],
    'host': creds["endpoint"],
    'database': creds["db_name"]
}

# connecting to endpoint with unpacking ( ⌐■-■)
conn = connector.connect(**config)

if conn.is_connected():
    cursor = conn.cursor()

<u>Task 1a.</u>
Execute a single query to return and print out the total assets owned by the bank (i.e., a cumulative total of the assets at all branches)

In [2]:
query = """SELECT SUM(assets) FROM branch;"""
cursor.execute(query)
result = cursor.fetchone()
print(f"bank's total assets: ${result[0]:,}")

bank's total assets: $24,600,480.0


<u>Task 1b.</u>

Create a trigger named `maintain_branch_size` on your branch table that ensures that a small branch does not become a large branch (i.e., that its total assets never get to 3 million dollars). Whenever an update to the assets of a small branch is attempted that would cause it to become a large branch, your trigger should store this excess money in a temporary table (e.g. `bailout_cache`) to be later divided equally among the existing large branches, capping the small branch at 2,999,999.99 in total assets.



In [7]:
query = """
CREATE TRIGGER maintain_branch_size BEFORE UPDATE ON branch
FOR EACH ROW
BEGIN
    -- make field to put excess assets
    DECLARE excess_assets NUMERIC(12, 2);
    
    -- check if the money is greater than the cap
    IF (OLD.assets < 3000000.00) AND (NEW.assets > 2999999.99) THEN
        
        -- getting excess by subtracting the amount that is over the cap
        SET excess_assets = NEW.assets - 2999999.99;
        
        -- set branch at cap
        SET NEW.assets = 2999999.99;
        
        -- add to buffer
        INSERT INTO bailout_cache (branch_name, assets)
        VALUES (OLD.branch_name, excess_assets)
        ON DUPLICATE KEY UPDATE assets = assets + excess_assets;
    END IF;
END;
"""
# query = """DROP TRIGGER maintain_branch_size"""
cursor.execute(query, multi=True)
conn.commit()

In [ ]:
CREATE TRIGGER maintain_branch_size BEFORE UPDATE ON branch
FOR EACH ROW
BEGIN
    -- make field to put excess assets
    DECLARE excess_assets NUMERIC(12, 2);
    -- check if the money is greater than the cap
    IF (NEW.assets) > 2999999.99 THEN
        -- getting excess by subtracting the amount that is over the cap
        SET excess_assets = NEW.assets - 2999999.99;
        -- set at cap
        SET NEW.assets = 2999999.99;
        -- add to buffer
        INSERT INTO bailout_cache (branch_name, assets)
        VALUES (OLD.branch_name, excess_assets)
        ON DUPLICATE KEY UPDATE assets = assets + excess_assets;
    END IF;
END;


<u>Task 1c.</u>

Execute a query that attempts to divide the bailout money ($10M) among the small branches as a test of your trigger.

In [5]:
bailout_per_branch = round(10000000.00 / 9, 2)
query = f"""
UPDATE branch SET assets = assets + {bailout_per_branch};
"""
cursor.execute(query)


In [6]:
query = "SELECT * FROM branch"
cursor.execute(query)
results = cursor.fetchall()
for row in results:
    print(row)

('Brighton', 'Brooklyn', 8111110.0)
('Central', 'Rye', 1511390.0)
('Downtown', 'Brooklyn', 2011110.0)
('Mianus', 'Horseneck', 1511310.0)
('North Town', 'Rye', 4811110.0)
('Perryridge', 'Horseneck', 2811110.0)
('Pownal', 'Bennington', 1511110.0)
('Redwood', 'Palo Alto', 3211110.0)
('Round Hill', 'Horseneck', 9111110.0)


<u>Task 1d.</u>

Execute a query that divides any excess money from the bailout (money in your bailout_cache table) among the large branches.

<u>Task 1e.</u>

Execute a single query to return and print out the total assets owned by the bank ( i.e., a cumulative total of the assets at all branches) after all changes above have been made.